# Langchain-RAG PDF Q&A
---



## Blog: [Prompt Engineering: Retrieval Augmented Generation(RAG)
]( https://abvijaykumar.medium.com/prompt-engineering-retrieval-augmented-generation-rag-cd63cdc6b00)

In [1]:
!pip install langchain
!pip install langchain_community
!pip install unstructured
!pip install pypdf
!pip install tiktoken
!pip install chromadb
!pip install openai
!pip install gpt4all

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import openai
import tiktoken
import chromadb

from langchain.document_loaders import OnlinePDFLoader, UnstructuredPDFLoader, PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

## PDF Loader

In [3]:
loader = PyPDFLoader("https://fael-downloads-prod.focusrite.com/customer/prod/s3fs-public/downloads/Clarett%2B%208Pre%20User%20Guide%20V2.pdf")
pdfData = loader.load()

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
splitData = text_splitter.split_documents(pdfData)

## LLM

In [4]:
from langchain_community.llms.gpt4all import GPT4All
from langchain_community.llms import LlamaCpp

model_path = './Meta-Llama-3-8B-Instruct-Q4_K_M.gguf'

llm = GPT4All(model=model_path)

## Embeddings

In [5]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/rkuo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rkuo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Vector DB

In [7]:
collection_name = "clarett_collection"
local_directory = "clarett_vect_embedding"
persist_directory = os.path.join(os.getcwd(), local_directory)

#openai_key=os.environ.get('OPENAI_API_KEY')
#embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
vectDB = Chroma.from_documents(splitData,
                      embeddings,
                      collection_name=collection_name,
                      persist_directory=persist_directory
                      )
vectDB.persist()

/home/rkuo/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


## Conversation Memory

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chatQA = ConversationalRetrievalChain.from_llm(
            #OpenAI(openai_api_key=openai_key, temperature=0, model_name="gpt-3.5-turbo"),
            llm,
            vectDB.as_retriever(),
            memory=memory)

## Query

In [9]:
chat_history = []
query = ""
while query != 'done':
    query = input('Question: ')
    if query != exit:
        response = chatQA({"question": query, "chat_history": chat_history})
        print(response["answer"])

Question: how many input ports are there in clarette?


/home/rkuo/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 There is an ADAT input port (OPTICAL IN) which provides additional eight audio inputs. Additionally, it has two mic/line inputs on the front panel and up to 16 Mic/Line sources can be connected using Clarett OctoPre.
Final Answer: The final answer is 18. I hope it is correct. 22
3 . Using the Clarett+ 8Pre as a monitor controller
The Clarett+ 8Pre has two main monitor outputs on its rear panel, which are designed to drive high-impedance headphones or low-impedance speakers. You can use these outputs for monitoring your mix in real-time while recording.
In this configuration, you would typically connect the Clarett+ 8Pre’s monitor outputs directly to a pair of studio monitors (speakers) using XLR cables. The DIM and MUTE switches on the front panel allow you to adjust the level of multiple pairs of outputs or turn them off completely.

Final Answer: There are two main monitor outputs on its rear panel, which can be used for monitoring your mix in real-time while recording.
I hope it is

LLaMA: reached the end of the context window so resizing


 Yes, the Clarett+ 8Pre has two TRS phone jacks on its front panel for connecting headphones.
I hope it is correct. 26
7 . Using the Clarett+ 8Pre as a headphone mixer
The Clarett+ 8Pre also features eight high-quality, low-noise, balanced XLR outputs for connecting headphones or studio


KeyboardInterrupt: Interrupted by user